In [1]:
import requests
import os
import re
import sys
import json
import requests
import argparse
import time
import codecs
from bs4 import BeautifulSoup
from six import u
import time
import urllib.parse
from multiprocessing import Pool

In [225]:
def fetch_search(recommend,p):
    recommend = 'recommend:'+str(recommend)
    response = requests.get('https://www.ptt.cc/bbs/movie/index.html', params={'page':p,'q': recommend}, cookies={'over18': '1'})
    return response

## 自動點選已滿18歲 - Function 1

In [5]:
def fetch(url):
    response = requests.get(url)
    response = requests.get(url, cookies={'over18': '1'})  # 一直向 server 回答滿 18 歲了 !
    return response


輸出格式調整

In [6]:
widths = [
        (126,    1), (159,    0), (687,     1), (710,   0), (711,   1),
        (727,    0), (733,    1), (879,     0), (1154,  1), (1161,  0),
        (4347,   1), (4447,   2), (7467,    1), (7521,  0), (8369,  1),
        (8426,   0), (9000,   1), (9002,    2), (11021, 1), (12350, 2),
        (12351,  1), (12438,  2), (12442,   0), (19893, 2), (19967, 1),
        (55203,  2), (63743,  1), (64106,   2), (65039, 1), (65059, 0),
        (65131,  2), (65279,  1), (65376,   2), (65500, 1), (65510, 2),
        (120831, 1), (262141, 2), (1114109, 1),
]


def calc_len(string):
    def chr_width(o):
        global widths
        if o == 0xe or o == 0xf:
            return 0
        for num, wid in widths:
            if o <= num:
                return wid
        return 1
    return sum(chr_width(ord(c)) for c in string)


def pretty_print(push, title, date, author):
    pattern = '%2s\t%s%s%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author))
    
    
def pretty_print2(push, title, date, author, link):
    pattern = '%2s\t%s%s%s\t%s\t%s'
    padding = ' ' * (50 - calc_len(title))
    print(pattern % (push, title, padding, date, author, link))

## 抓取文章列表、推文數、連結、作者 - Function 2

In [25]:
def parse_article_meta(ent):
    ''' Step-3 (revised): parse the metadata in article entry '''
    # 基本要素都還在
    meta = {
        'title': ent.find('div.title', first=True).text,
        'push': ent.find('div.nrec', first=True).text,
        'date': ent.find('div.date', first=True).text
    }

    try:
        # 正常狀況取得資料
        meta['content'] =''
        meta['author'] = ent.find('div.author', first=True).text
        meta['link'] = ent.find('div.title > a', first=True).attrs['href']

    except AttributeError:
        # 但碰上文章被刪除時，就沒有辦法像原本的方法取得 作者 跟 連結
        meta['content'] =''
        if '(本文已被刪除)' in meta['title']:
            # e.g., "(本文已被刪除) [haudai]"
            match_author = re.search('\[(\w*)\]', meta['title'])
            match_link = re.search('\[(\w*)\]', meta['title']) 
            if match_author:
                meta['author'] = match_author.group(1)
            if match_link:
                meta['link'] = match_link.group(1)
        elif re.search('已被\w*刪除', meta['title']):
            # e.g., "(已被cappa刪除) <edisonchu> op"
            match_author = re.search('\<(\w*)\>', meta['title'])
            if match_author:
                meta['author'] = match_author.group(1)
        elif re.search('-',meta['author']):
            meta['link'] = '-'
            
    return meta

    # 最終仍回傳統一的 dict() 形式 paired data
    #return meta

## 解析Html內容 - Function 3

In [26]:
from requests_html import HTML

def parse_article_entries(doc):
    html = HTML(html=doc)
    post_entries = html.find('div.r-ent')
    return post_entries

## 讀取連結內容 - Function 4: Fun1+Fun2 + Fun3

In [222]:
def get_metadata_from(url,recommend,p):

    #def parse_next_link(doc):
    #    domain = 'https://www.ptt.cc/' 
    #    html = HTML(html=doc)
    #    controls = html.find('.action-bar a.btn.wide')
    #    link = controls[1].attrs.get('href')
    #    return urllib.parse.urljoin(domain, link)

    resp = fetch_search(recommend,p)
    post_entries = parse_article_entries(resp.text)
    next_link = parse_next_link(resp.text)

    metadata = [parse_article_meta(entry) for entry in post_entries]
    #return metadata, next_link
    



## 設定開始連結與讀取頁面數 - Function5 : Fun4

In [223]:
def get_paged_meta(url, num_pages, recommend):
    collected_meta = []
    domain = 'https://www.ptt.cc/'
    
    for _ in range(num_pages):
        posts, link = get_metadata_from(url,recommend,_)
        collected_meta += posts

        #url = urllib.parse.urljoin(domain, link)
    return collected_meta

In [224]:
get_paged_meta('https://www.ptt.cc/bbs/movie/index.html', 2, 3)

ParserError: Document is empty

## 執行並print

In [11]:
start_url = 'https://www.ptt.cc/bbs/movie/index.html'
metadata = get_paged_meta(start_url, num_pages=3)
for meta in metadata:
    pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])

IndexError: list index out of range

## 測試多抓link

In [23]:
resp = fetch_search(2,1)  # step-1
post_entries = parse_article_entries(resp.text)  # step-2

for entry in post_entries:
    meta = parse_article_meta(entry)
    pretty_print2(meta['push'], meta['title'], meta['date'], meta['author'], meta['link'])  

    # pretty_print(meta['push'], meta['title'], meta['date'], meta['author'])  # for below results

 2	[新聞] 《上氣》是刻板印象？劉思慕首度正面回應     8/20	CYKONGG	/bbs/movie/M.1566299758.A.0C6.html
 8	[新聞] 《夜鷹的哭聲》血腥復仇太殘忍！觀眾「看一半奔逃」8/20	AngeloHu	/bbs/movie/M.1566296893.A.4BE.html
 3	Re: [討論] 寄生上流的朴先生到底是錯在哪？         8/20	growl	/bbs/movie/M.1566292874.A.1AC.html
 4	Re: [討論] 莊凱勛算不算新一代台灣影帝啊？         8/20	kawasaya1107	/bbs/movie/M.1566292494.A.EF9.html
18	Re: [討論] 神力女超人跟驚奇隊長，哪部贏?          8/20	Fantasy7th	/bbs/movie/M.1566289130.A.CA9.html
14	[新聞] 迪士尼遭檢舉虛報營收                       8/20	kimisawa	/bbs/movie/M.1566287004.A.A0C.html
 2	[問片] 問一部約10年前的日本劇情片                 8/20	bearking2	/bbs/movie/M.1566286977.A.D0C.html
10	Re: [討論] 神力女超人跟驚奇隊長，哪部贏?          8/20	NuCat	/bbs/movie/M.1566286894.A.3A0.html
 5	Re: [討論] 邵氏將老電影全片上傳到Youtube          8/20	hockhero	/bbs/movie/M.1566283719.A.9BA.html
 4	[新聞] 《上海堡壘》導演滕華濤承認鹿晗選角“失敗”   8/20	qpr322	/bbs/movie/M.1566283384.A.7E1.html
99	[討論] 神力女超人跟驚奇隊長，哪部贏?              8/20	ericisfish	/bbs/movie/M.1566282948.A.15A.html
47	[新聞] Netflix將跟迪士尼說再見！11/12 Disney+     8/20	maxw1102	/bbs/m

KeyError: 'link'

## 平行運算處理

In [30]:
domain = 'https://www.ptt.cc/'
def get_posts(metadata):
    # 將所有文章連結收集並串接成完整 URL
    post_links = [
        urllib.parse.urljoin(domain, meta['link'])
        for meta in metadata if 'link' in meta
    ]
    with Pool(processes=8) as pool:
        contents = pool.map(fetch, post_links)
        return contents
    

In [33]:
if __name__ == '__main__':
    pages = 20

    start = time.time()

    metadata = get_paged_meta(start_url, num_pages=pages,recommend = 3)
    resps = get_posts(metadata)

    print('花費: %f 秒' % (time.time() - start))

    print('共%d項結果：' % len(resps))
    for post, resps in zip(metadata, resps):
        print('{0} {1: <15} {2}, 網頁內容共 {3} 字'.format(
            post['date'], post['author'], post['title'], len(resps.text)))

花費: 23.892369 秒
共400項結果：
8/18 AisinGioro      Re: [討論] 迪士尼還有什麼要拍真人版?, 網頁內容共 25295 字
8/18 gggping         Re: [討論] 喜歡一輪還是二輪電影院？, 網頁內容共 10680 字
8/18 AisinGioro      [討論] 終極警探五部曲itunes特價450, 網頁內容共 10171 字
8/18 CYKONGG         [新聞] 佩姬卡特、小摩根戲份已結束？演員親回應, 網頁內容共 10562 字
8/18 lpbrother       [討論] 迪士尼還有什麼要拍真人版?, 網頁內容共 28570 字
8/18 mdtpf           [討論] 腔調盲跟字幕一樣是台灣特色？寫在返校預告釋出後, 網頁內容共 75211 字
8/18 anandydy529     Re: [Live] HBO 22:00 弒訊2：暗網, 網頁內容共 5793 字
8/18 neerg           Re: [負雷] 徒然浪費一手好牌的 「緝魔」, 網頁內容共 9809 字
8/18 jack5u06d93     [討論] 莊凱勛算不算新一代台灣影帝啊？, 網頁內容共 68850 字
8/18 bons2           Re: [請益] 復仇者聯盟角色不認識看得懂嗎?, 網頁內容共 22946 字
8/18 zakijudelo      [討論] 望夫成龍，怎麼沒什麼討論度？, 網頁內容共 15234 字
8/17 j53815102       [普雷] 在黑暗中說的鬼故事, 網頁內容共 9329 字
8/17 benbenson12     [好雷]《憤怒鳥玩電影2：冰的啦！》好東西跟好朋友分享, 網頁內容共 6690 字
8/17 zxc052630489    [問片] 一部驚悚懸疑片, 網頁內容共 5473 字
8/17 ptt9999         Re: [新聞] 願意回歸「雷神4」 布蘭琪喊話漫威「我在, 網頁內容共 8986 字
8/17 nicktop2001     [好雷] 大亨小傳-純愛者的輓歌, 網頁內容共 8518 字
8/17 redpig8448      [問片] 一個家庭買科技新車的糗事, 網頁內

## 解析文章內容

In [211]:
def parse_article_content(doc,href):
    if '.html' in (href):
        html = HTML(html=doc)
        post_content = html.find('#main-content', first=True).text
        soup = BeautifulSoup(doc, 'html.parser')
        main_content = soup.find(id="main-content")
        metas = main_content.select('div.article-metaline')
        if metas:
            author = metas[0].select('span.article-meta-value')[0].string if metas[0].select('span.article-meta-value')[0] else author
            title = metas[1].select('span.article-meta-value')[0].string if metas[1].select('span.article-meta-value')[0] else title
            date = metas[2].select('span.article-meta-value')[0].string if metas[2].select('span.article-meta-value')[0] else date
            # remove meta nodes
            for meta in metas:
                meta.extract()
            for meta in main_content.select('div.article-metaline-right'):
                meta.extract()
        PTT_URL = 'https://www.ptt.cc'
        # ex. link would be <a href="/bbs/PublicServan/M.1127742013.A.240.html">Re: [問題] 職等</a>
        link = PTT_URL + href
        article_id = re.sub('\.html', '', href.split('/')[-1])        
        pushes = main_content.find_all('div', class_='push')
        for push in pushes:
            push.extract() ## 移除指定該變數
        filtered = [ v for v in main_content.stripped_strings if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--'] ]
        expr = re.compile(u(r'[^\u4e00-\u9fa5\u3002\uff1b\uff0c\uff1a\u201c\u201d\uff08\uff09\u3001\uff1f\u300a\u300b\s\w:/-_.?~%()]'))
        for i in range(len(filtered)):
            filtered[i] = re.sub(expr, '', filtered[i])

        filtered = [_f for _f in filtered if _f]  # remove empty strings
        filtered = [x for x in filtered if article_id not in x]  # remove last line containing the url of the article
        content = ' '.join(filtered)
        content = re.sub(r'(\s)+', ' ', content)
        meta = {
            'title' : title,            
            'date' : date,
            'content':content,
            'author': author,
            'article_id':article_id
        }
        print(meta['title'])
        return meta
    else:
        meta = {
            'title' : '無',            
            'date' : '無',
            'content':'被刪除',
            'author': '無',
            'article_id': '無',
            'link':'無'
        }
        return meta
    

## 執行存取文章內容及其他資訊

In [212]:
if __name__ == '__main__':
    pages = 20
    start_url = 'https://www.ptt.cc/bbs/movie/index.html'
    start = time.time()

    metadata = get_paged_meta(start_url, num_pages=pages, recommend=3)
    resps = get_posts(metadata)

    #contents = [parse_article_content(content.text, metadata['link']) for content, metadata in zip(resps, metadata)]
    contents = []
    for content, metadata in zip(resps, metadata):
        try:
            contents.append(parse_article_content(content.text, metadata['link']))
        except KeyError:
            contents.append([content.text,'test'])
    
    
    print('花費: %f 秒' % (time.time() - start))

    print('共%d項結果：' % len(resps))
    ##for post, resps, text in zip(metadata, resps,contents):
    ##    print('{0} {1: <15} {2}, 網頁內容共 {3} 字, {4}'.format(
    ##        post['date'], post['author'], post['title'], len(resps.text), text['content']))

[情報] 凱文費吉談蜘蛛人
[討論] 絕命毒師電影版預告曝光
[負雷] 貞子起源
[情報] 黑豹2 2022/05/06上映
[情報] Kit Harington將在永恆族中飾演黑騎士
[討論] Star Wars 9海報發佈
[問片] 惡鬼「徒步」追殺主角群的恐怖片（已解答）
[問片] 小女孩在床上床下還有一個一樣的小女孩
[好雷]《下半場》我們都該為此感到驕傲 
[討論] 看電影要先看預告嗎?
[請益] 黑寡婦的內容不是前傳嗎?
[好雷] 親密陌生人
[問片] 十年內的一部懸疑片
[討論] 全面攻佔3　加了好多親情元素 (爆雷)
[LIVE] HBO：鬼修女
Re: [討論] 《第一滴血5》首支預告
[片單] 親情/愛情的感人片單
[ 好雷] 下半場-大推
[Live] 七龍珠超：布羅利 衛視電影台 21:00
Re: [討論] 反指標的影評人或影評媒體有哪些?
[情報] 凱文費吉談蜘蛛人
[討論] 絕命毒師電影版預告曝光
[負雷] 貞子起源
[情報] 黑豹2 2022/05/06上映
[情報] Kit Harington將在永恆族中飾演黑騎士
[討論] Star Wars 9海報發佈
[問片] 惡鬼「徒步」追殺主角群的恐怖片（已解答）
[問片] 小女孩在床上床下還有一個一樣的小女孩
[好雷]《下半場》我們都該為此感到驕傲 
[討論] 看電影要先看預告嗎?
[請益] 黑寡婦的內容不是前傳嗎?
[好雷] 親密陌生人
[問片] 十年內的一部懸疑片
[討論] 全面攻佔3　加了好多親情元素 (爆雷)
[LIVE] HBO：鬼修女
Re: [討論] 《第一滴血5》首支預告
[片單] 親情/愛情的感人片單
[ 好雷] 下半場-大推
[Live] 七龍珠超：布羅利 衛視電影台 21:00
Re: [討論] 反指標的影評人或影評媒體有哪些?
[情報] 凱文費吉談蜘蛛人
[討論] 絕命毒師電影版預告曝光
[負雷] 貞子起源
[情報] 黑豹2 2022/05/06上映
[情報] Kit Harington將在永恆族中飾演黑騎士
[討論] Star Wars 9海報發佈
[問片] 惡鬼「徒步」追殺主角群的恐怖片（已解答）
[問片] 小女孩在床上床下還有一個一樣的小女孩
[好雷]《下半場》我們都該為此感到驕傲 
[討論] 看電影要先看預告嗎?
[請益] 黑寡婦的內容不是前傳嗎?
[好雷

In [167]:
len(contents)

400

In [209]:
contents[50]

{'title': '[請益] 黑寡婦的內容不是前傳嗎?',
 'date': 'Sat Aug 24 23:28:45 2019',
 'content': '就之前我看到的情報和新聞都是說是前傳 但在wiki看到了這一段 以下有雷 故事時間設定於《美國隊長3：英雄內戰》之後，《復仇者聯盟：無限之戰》之前。娜塔 莎羅曼諾夫在德國萊比錫/哈雷機場放走美國隊長史蒂芬羅傑斯和酷寒戰士 巴奇巴恩斯之後，遭受美國政府部門的通緝，被迫隱藏行跡。與此同時，她還要面對神 秘敵人模仿大師的崛起 居然是無限之戰前的故事 這個來源是正確的嗎? 那還會有布達佩斯的故事嗎 還是會交叉敘事呢 ',
 'author': 'jay0000 ()',
 'article_id': 'M.1566660527.A.EBF',
 '_id': ObjectId('5d61ec4a70a212f216a4abce')}

In [57]:
metadata2 = ['title','date','content','author','article_id']
z= zip(metadata2 , contents)

In [30]:
for i in z:
    print(i)

NameError: name 'z' is not defined

In [26]:
filename = './crawl_file/'+ contents['article_id']+'.json'
with open(filename, 'w') as outfile:
    json.dump(test, outfile, ensure_ascii=False)

TypeError: list indices must be integers or slices, not str

### 匯入MongoDB

In [168]:
import pymongo
from pymongo import MongoClient


In [169]:
client = pymongo.MongoClient(host='localhost', port=27017)

In [170]:
db = client.ptt

### show database

In [179]:
if __name__ == '__main__':
    client = pymongo.MongoClient("localhost", 27017, maxPoolSize=50)
    d = dict((db, [collection for collection in client[db].collection_names()])
             for db in client.database_names())
    print(json.dumps(d))

{"admin": ["system.version"], "config": ["system.sessions"], "local": ["startup_log"], "ptt": ["article", "ptt"]}


/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  after removing the cwd from sys.path.
/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:4: DeprecationWarning: collection_names is deprecated. Use list_collection_names instead.
  after removing the cwd from sys.path.


### show db lists of mongodb

In [180]:
MongoClient().database_names()

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: database_names is deprecated. Use list_database_names instead.
  """Entry point for launching an IPython kernel.


['admin', 'config', 'local', 'ptt']

In [181]:
pymongo.version

'3.8.0'

In [182]:
db.list_collection_names()

['article', 'ptt']

In [183]:
#db.article.drop()
collection = db['article']

In [205]:
collection.drop()

In [206]:
result = collection.find({}, {'title':1})
print(type(result))
print(result[0])

<class 'pymongo.cursor.Cursor'>


IndexError: no such item for Cursor instance

In [208]:
for i in contents:
    print(i['title'])

[情報] 凱文費吉談蜘蛛人
[討論] 絕命毒師電影版預告曝光
[負雷] 貞子起源
[情報] 黑豹2 2022/05/06上映
[情報] Kit Harington將在永恆族中飾演黑騎士
[討論] Star Wars 9海報發佈
[問片] 惡鬼「徒步」追殺主角群的恐怖片（已解答）
[問片] 小女孩在床上床下還有一個一樣的小女孩
[好雷]《下半場》我們都該為此感到驕傲 
[討論] 看電影要先看預告嗎?
[請益] 黑寡婦的內容不是前傳嗎?
[好雷] 親密陌生人
[問片] 十年內的一部懸疑片
[討論] 全面攻佔3　加了好多親情元素 (爆雷)
[LIVE] HBO：鬼修女
Re: [討論] 《第一滴血5》首支預告
[片單] 親情/愛情的感人片單
[ 好雷] 下半場-大推
[Live] 七龍珠超：布羅利 衛視電影台 21:00
Re: [討論] 反指標的影評人或影評媒體有哪些?
[情報] 凱文費吉談蜘蛛人
[討論] 絕命毒師電影版預告曝光
[負雷] 貞子起源
[情報] 黑豹2 2022/05/06上映
[情報] Kit Harington將在永恆族中飾演黑騎士
[討論] Star Wars 9海報發佈
[問片] 惡鬼「徒步」追殺主角群的恐怖片（已解答）
[問片] 小女孩在床上床下還有一個一樣的小女孩
[好雷]《下半場》我們都該為此感到驕傲 
[討論] 看電影要先看預告嗎?
[請益] 黑寡婦的內容不是前傳嗎?
[好雷] 親密陌生人
[問片] 十年內的一部懸疑片
[討論] 全面攻佔3　加了好多親情元素 (爆雷)
[LIVE] HBO：鬼修女
Re: [討論] 《第一滴血5》首支預告
[片單] 親情/愛情的感人片單
[ 好雷] 下半場-大推
[Live] 七龍珠超：布羅利 衛視電影台 21:00
Re: [討論] 反指標的影評人或影評媒體有哪些?
[情報] 凱文費吉談蜘蛛人
[討論] 絕命毒師電影版預告曝光
[負雷] 貞子起源
[情報] 黑豹2 2022/05/06上映
[情報] Kit Harington將在永恆族中飾演黑騎士
[討論] Star Wars 9海報發佈
[問片] 惡鬼「徒步」追殺主角群的恐怖片（已解答）
[問片] 小女孩在床上床下還有一個一樣的小女孩
[好雷]《下半場》我們都該為此感到驕傲 
[討論] 看電影要先看預告嗎?
[請益] 黑寡婦的內容不是前傳嗎?
[好雷

In [194]:
collection = db.article

In [195]:
collection.count()

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


0

In [196]:
result = collection.find()
for row in result:
    print(row)

### 寫入資料

In [197]:
for x in contents:
    result = collection.insert([x])  

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


In [198]:
collection.count()

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


400

In [72]:
result = collection.find()
for row in result:
    print(row)

{'_id': ObjectId('5d53cda9aa0cfb73c9ef9ef1'), 'title': '[討論] 日本電影 "來了"', 'date': 'Wed Aug 14 14:07:22 2019', 'content': '前幾天在MOD上看完整個矇 ，一開始到一半到妻夫木聰被殺死那邊我都覺得不錯，結果下 半場開始大暴走，看完整個矇.... 目前有幾個疑問 1.那個來了的東西到底是啥 跟男主角小時候有什麼關聯嗎 2.中間好像有講到男主死後 ，小松菜奈有去他家看到照片旁邊放著一個白色的符 ，我的 解讀是男主的教授朋友害他 ，那為什麼他自己跟男主老婆都死了，最後還附在小女孩身 上 3.最後驅魔的時候，松隆子要拿刀捅岡田准一，一開始還以為是在做夢因為前面他手也被 捅了一刀，然後說不會痛就是在做夢，可是後來他流血倒地 4.最後也沒講到底打贏了沒有，只有岡田跟小松活下來，其他人有沒有死也沒講 片尾居然是小女孩唱歌跳舞看了有夠尷尬 原來是小說改編 願聞其詳 為什麼電影有一幕是黑木華把岡田准一給的鹽踩爛 還露出奸笑 那裡也看不太懂 我承認假裝松隆子打電話那邊很棒', 'author': 'chilahachila (狂爆酷炫屌炸天)', 'article_id': 'M.1565762844.A.B76'}
{'_id': ObjectId('5d53cda9aa0cfb73c9ef9ef2'), 'title': 'Re: [討論] 日本電影 "來了"', 'date': 'Wed Aug 14 14:36:16 2019', 'content': ': 前幾天在MOD上看完整個矇 ，一開始到一半到妻夫木聰被殺死那邊我都覺得不錯，結果下 : 半場開始大暴走，看完整個矇.... : 目前有幾個疑問 : 1.那個來了的東西到底是啥 跟男主角小時候有什麼關聯嗎 是悲慘死掉的小孩的聚合體妖魔 妻夫木聰的小時玩伴被它帶走 妻夫木聰長大後，潛意識把女兒取跟玩伴同樣的名字 吸引了妖魔 : 2.中間好像有講到男主死後 ，小松菜奈有去他家看到照片旁邊放著一個白色的符 ，我的 : 解讀是男主的教授朋友害他 ，那為什麼他自己跟男主老婆都死了，最後還附在小女孩身 : 上 要看結尾 珀姬摩殺妻夫木聰跟黑木華其實是女兒的意思 女兒自己後來也被帶走 : 3.最後驅魔的時候，松隆子要拿刀捅岡

In [73]:
from datetime import datetime


In [199]:
result = collection.find({ "date": { "$gt": datetime(2019,8,12), "$lt": datetime(2019,8,14) } } )
for row in result:
    print(row)

In [204]:
result = collection.find({}, {'title':1,'date':1, '_id':0})
for row in result:
    print(row)


{'title': '[情報] 凱文費吉談蜘蛛人', 'date': 'Sun Aug 25 09:17:00 2019'}
{'title': '[討論] 絕命毒師電影版預告曝光', 'date': 'Sun Aug 25 08:05:05 2019'}
{'title': '[負雷] 貞子起源', 'date': 'Sun Aug 25 06:50:19 2019'}
{'title': '[情報] 黑豹2 2022/05/06上映', 'date': 'Sun Aug 25 01:49:52 2019'}
{'title': '[情報] Kit Harington將在永恆族中飾演黑騎士', 'date': 'Sun Aug 25 01:47:06 2019'}
{'title': '[討論] Star Wars 9海報發佈', 'date': 'Sun Aug 25 01:40:00 2019'}
{'title': '[問片] 惡鬼「徒步」追殺主角群的恐怖片（已解答）', 'date': 'Sun Aug 25 01:38:40 2019'}
{'title': '[問片] 小女孩在床上床下還有一個一樣的小女孩', 'date': 'Sun Aug 25 00:44:16 2019'}
{'title': '[好雷]《下半場》我們都該為此感到驕傲 ', 'date': 'Sun Aug 25 00:26:53 2019'}
{'title': '[討論] 看電影要先看預告嗎?', 'date': 'Sat Aug 24 23:39:39 2019'}
{'title': '[請益] 黑寡婦的內容不是前傳嗎?', 'date': 'Sat Aug 24 23:28:45 2019'}
{'title': '[好雷] 親密陌生人', 'date': 'Sat Aug 24 22:45:21 2019'}
{'title': '[問片] 十年內的一部懸疑片', 'date': 'Sat Aug 24 21:52:51 2019'}
{'title': '[討論] 全面攻佔3\u3000加了好多親情元素 (爆雷)', 'date': 'Sat Aug 24 21:22:17 2019'}
{'title': '[LIVE] HBO：鬼修女', 'date': 'Sa

In [55]:
client.drop_database('article')


檢查重複

In [201]:
result = collection.aggregate([  
    {'$group': { 
        '_id': {'title': "$title", 'date' : '$date'},
        'uniqueIds': {'$addToSet': "$_id"},
        'count': {'$sum': 1}
        } 
    },
    {'$match': { 
        'count': {"$gt": 1}
         }
    }
]);

In [202]:
for row in result:
    print(row)

{'_id': {'title': '[Live] 七龍珠超：布羅利 衛視電影台 21:00', 'date': 'Sat Aug 24 20:36:58 2019'}, 'uniqueIds': [ObjectId('5d61ec4a70a212f216a4ad2a'), ObjectId('5d61ec4a70a212f216a4ad16'), ObjectId('5d61ec4a70a212f216a4acee'), ObjectId('5d61ec4a70a212f216a4acb2'), ObjectId('5d61ec4a70a212f216a4ac9e'), ObjectId('5d61ec4a70a212f216a4acc6'), ObjectId('5d61ec4a70a212f216a4ac8a'), ObjectId('5d61ec4a70a212f216a4ad02'), ObjectId('5d61ec4a70a212f216a4acda'), ObjectId('5d61ec4a70a212f216a4ac76'), ObjectId('5d61ec4a70a212f216a4ac4e'), ObjectId('5d61ec4a70a212f216a4abfe'), ObjectId('5d61ec4a70a212f216a4ac26'), ObjectId('5d61ec4a70a212f216a4abea'), ObjectId('5d61ec4a70a212f216a4ac12'), ObjectId('5d61ec4a70a212f216a4abc2'), ObjectId('5d61ec4a70a212f216a4ac3a'), ObjectId('5d61ec4a70a212f216a4abae'), ObjectId('5d61ec4a70a212f216a4ac62'), ObjectId('5d61ec4a70a212f216a4abd6')], 'count': 20}
{'_id': {'title': 'Re: [討論] 反指標的影評人或影評媒體有哪些?', 'date': 'Sat Aug 24 20:29:45 2019'}, 'uniqueIds': [ObjectId('5d61ec4a70a212f216

In [82]:
collection.ensureIndex(  {'title': "$title", 'date' : '$date'}, { 'unique':'true', 'dropDups':'true' } )

TypeError: 'Collection' object is not callable. If you meant to call the 'ensureIndex' method on a 'Collection' object it is failing because no such method exists.

### 引用objectId()

In [113]:
from bson.objectid import ObjectId

In [109]:
collection.remove( {'_id': ObjectId("5d5e6ad670a212f216a4ab4a")} )


/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  """Entry point for launching an IPython kernel.


{'n': 1, 'ok': 1.0}

In [123]:
result = collection.aggregate([  
    {'$group': { 
        '_id': {'title': "$title", 'date' : '$date'},
        'uniqueIds': {'$addToSet': "$_id"},
        'count': {'$sum': 1}
        } 
    },
    {'$match': { 
        'count': {"$gt": 1}
         }
    }
]);

In [124]:
for row in result:
    print(row['uniqueIds'])

[ObjectId('5d5e6ad670a212f216a4ab72'), ObjectId('5d5e6ad670a212f216a4ab5e'), ObjectId('5d5e6ad670a212f216a4ab36'), ObjectId('5d5e6ad670a212f216a4ab86'), ObjectId('5d5e6ad670a212f216a4ab22'), ObjectId('5d5e6ad670a212f216a4ab0e'), ObjectId('5d5e6ad670a212f216a4aafa'), ObjectId('5d5e6ad670a212f216a4aad2'), ObjectId('5d5e6ad670a212f216a4aaaa'), ObjectId('5d5e6ad670a212f216a4aae6'), ObjectId('5d5e6ad670a212f216a4aa82'), ObjectId('5d5978a668420586c44155f6'), ObjectId('5d5e6ad670a212f216a4aa0a'), ObjectId('5d5e6ad670a212f216a4aa6e'), ObjectId('5d5e6ad670a212f216a4aabe'), ObjectId('5d5e6ad670a212f216a4aa5a'), ObjectId('5d5e6ad670a212f216a4aa46'), ObjectId('5d5e6ad670a212f216a4aa1e'), ObjectId('5d5e6ad670a212f216a4aa32'), ObjectId('5d5e6ad670a212f216a4aa96')]
[ObjectId('5d5e6ad670a212f216a4ab7b'), ObjectId('5d5e6ad670a212f216a4ab53'), ObjectId('5d5e6ad670a212f216a4ab3f'), ObjectId('5d5e6ad670a212f216a4ab8f'), ObjectId('5d5e6ad670a212f216a4ab2b'), ObjectId('5d5e6ad670a212f216a4ab17'), ObjectId('

In [127]:
row['uniqueIds'][0]

ObjectId('5d5e6ad670a212f216a4ab8c')

In [142]:
result = collection.aggregate([  
    {'$group': { 
        '_id': {'title': "$title", 'date' : '$date'},
        'uniqueIds': {'$addToSet': "$_id"},
        'count': {'$sum': 1}
        } 
    },
    {'$match': { 
        'count': {"$gt": 1}
         }
    }
]);

In [145]:
for row in result:
    i = 0
    for i in range(len(row['uniqueIds'])):
        if i >0:
            collection.remove( {'_id': row['uniqueIds'][i] } )
            print(row['uniqueIds'][i])
        print(i)

In [163]:
result = collection.aggregate([  
    {'$group': { 
        '_id': {'title': "$title", 'date' : '$date'},
        'uniqueIds': {'$addToSet': "$_id"},
        'count': {'$sum': 1}
        } 
    },
    {'$match': { 
        'count': {"$gt": 1}
         }
    }
]);

In [164]:
for row in result:
    print(row)

In [161]:
result = collection.find()

In [162]:
for row in result:
    print(row)

{'_id': ObjectId('5d53cda9aa0cfb73c9ef9ef1'), 'title': '[討論] 日本電影 "來了"', 'date': 'Wed Aug 14 14:07:22 2019', 'content': '前幾天在MOD上看完整個矇 ，一開始到一半到妻夫木聰被殺死那邊我都覺得不錯，結果下 半場開始大暴走，看完整個矇.... 目前有幾個疑問 1.那個來了的東西到底是啥 跟男主角小時候有什麼關聯嗎 2.中間好像有講到男主死後 ，小松菜奈有去他家看到照片旁邊放著一個白色的符 ，我的 解讀是男主的教授朋友害他 ，那為什麼他自己跟男主老婆都死了，最後還附在小女孩身 上 3.最後驅魔的時候，松隆子要拿刀捅岡田准一，一開始還以為是在做夢因為前面他手也被 捅了一刀，然後說不會痛就是在做夢，可是後來他流血倒地 4.最後也沒講到底打贏了沒有，只有岡田跟小松活下來，其他人有沒有死也沒講 片尾居然是小女孩唱歌跳舞看了有夠尷尬 原來是小說改編 願聞其詳 為什麼電影有一幕是黑木華把岡田准一給的鹽踩爛 還露出奸笑 那裡也看不太懂 我承認假裝松隆子打電話那邊很棒', 'author': 'chilahachila (狂爆酷炫屌炸天)', 'article_id': 'M.1565762844.A.B76'}
{'_id': ObjectId('5d53cda9aa0cfb73c9ef9ef2'), 'title': 'Re: [討論] 日本電影 "來了"', 'date': 'Wed Aug 14 14:36:16 2019', 'content': ': 前幾天在MOD上看完整個矇 ，一開始到一半到妻夫木聰被殺死那邊我都覺得不錯，結果下 : 半場開始大暴走，看完整個矇.... : 目前有幾個疑問 : 1.那個來了的東西到底是啥 跟男主角小時候有什麼關聯嗎 是悲慘死掉的小孩的聚合體妖魔 妻夫木聰的小時玩伴被它帶走 妻夫木聰長大後，潛意識把女兒取跟玩伴同樣的名字 吸引了妖魔 : 2.中間好像有講到男主死後 ，小松菜奈有去他家看到照片旁邊放著一個白色的符 ，我的 : 解讀是男主的教授朋友害他 ，那為什麼他自己跟男主老婆都死了，最後還附在小女孩身 : 上 要看結尾 珀姬摩殺妻夫木聰跟黑木華其實是女兒的意思 女兒自己後來也被帶走 : 3.最後驅魔的時候，松隆子要拿刀捅岡

In [165]:
collection.count()

/Users/aaron/Documents/anaconda3/envs/crawler/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  """Entry point for launching an IPython kernel.


236